In [5]:
from ase.io import read
from ase.visualize import view
import numpy as np

# V1

In [4]:
V1=read("./1_opt/CONTCAR")
del V1[168]
view(V1)

In [ ]:
from ase.io.vasp import write_vasp
write_vasp("V1_POSCAR",V1,label="changed",direct=True,)

# V2

In [18]:
V2=read("./1_opt/CONTCAR")
v1p=V2[168].position.copy()
del V2[168]
v2p=V2[168].position.copy()
V2[168].position=v1p
view(V2)

In [9]:
from ase.io.vasp import write_vasp
write_vasp("V2positive_POSCAR",V2,label="changed",direct=True,)

# V3

In [16]:
V3=read("V2positive_POSCAR")
V3[165].position=v2p
view(V3)

In [17]:
from ase.io.vasp import write_vasp
write_vasp("V3positive_POSCAR",V3,label="changed",direct=True,)

# neb V1 to V2

In [6]:
import math
def move_near(p0,p1,cell):
    # p0,p1 np.array((n,3)) positions in a cell
    # cell np.array((3,3)) cell vectors
    # return np.array(n,3) positions of p1 after moving
    
    def period(p,a,b,c):
        return p+a*cell[0]+b*cell[1]+c*cell[2]
    
    r=np.empty((p0.shape[0],3))
    tmp_d=np.empty((p0.shape[0],27))

    n_ijk=np.empty((27,3))
    n=0
    for i in [-1,0,1]:
        for j in [-1,0,1]:
            for k in [-1,0,1]:
                tmp_d[:,n]=np.sqrt(np.square(period(p1,i,j,k)-p0).sum(1))
                n_ijk[n]=np.array([i,j,k])
                n=n+1
    best=tmp_d.argmin(1)
    for i in range(p0.shape[0]):
        n=best[i]
        r[i,:]=period(p1[i],n_ijk[n,0],n_ijk[n,1],n_ijk[n,2])
        
    return r

def rotate(pr,k,theta,p0=np.zeros(3)):
    # rotate a group of points along specific axis counterclockwise
    # pr np.array(shape=(N,3)) points to be rotated
    # k np.array(shape=3) rotation axis
    # theta float or int rotation rad
    # p0 np.array(shape=3) grid origin
    # return np.array(shape=(N,3)) points after rotation
    if type(pr)!=np.ndarray:
        pr=np.array(pr)
    if type(k)!=np.ndarray:
        k=np.array(k)
    if type(p0)!=np.ndarray:
        p0=np.array(p0)
    k=k/np.sqrt((k*k).sum())
    p_opt=(pr-p0).T
    cos=math.cos(theta)
    sin=math.sin(theta)
    m=np.zeros((3,3))
    m[0,0]=cos+k[0]*k[0]*(1-cos)
    m[0,1]=-sin*k[2]+(1-cos)*k[0]*k[1]
    m[0,2]=sin*k[1]+(1-cos)*k[0]*k[2]
    m[1,0]=sin*k[2]+(1-cos)*k[0]*k[1]
    m[1,1]=cos+k[1]*k[1]*(1-cos)
    m[1,2]=-sin*k[0]+(1-cos)*k[1]*k[2]
    m[2,0]=-sin*k[1]+(1-cos)*k[0]*k[2]
    m[2,1]=sin*k[0]+(1-cos)*k[1]*k[2]
    m[2,2]=cos+k[2]*k[2]*(1-cos)
    return np.dot(m,p_opt).T+p0

In [7]:
V1=read("./5_V1positive/CONTCAR")
V2=read("./6_V2positive/CONTCAR")
V2.positions=move_near(V1.positions,V2.positions,V1.cell)
view(V1)
view(V2)

In [11]:
n_img=4
img_list=[]
for i in range(n_img):
    img_list.append(V1.copy())
    img_list[i].positions=V1.positions+(i+1)/(n_img+1)*(V2.positions-V1.positions)
    
d1=np.sqrt(np.square(V1[168].position-V1[185].position).sum())
d2=np.sqrt(np.square(V2[168].position-V2[185].position).sum())
print(d1,d2)
for i in range(n_img):
    vec=img_list[i][168].position-img_list[i][185].position
    vec=vec/np.linalg.norm(vec)*(d1*(i+1)/(n_img+1)+d2*(n_img-i)/(n_img+1))
    img_list[i][168].position=img_list[i][185].position+vec

view(img_list[3])

3.165118981943951 3.197487468474319


In [12]:
from ase.io.vasp import write_vasp
import os

dir_list=[]
for i in range(n_img+2):
    if i<10:
        dir_name="0"+str(i)
    else:
        dir_name=str(i)
    dir_list.append(dir_name)
    if not os.path.exists("./8_nebV1V2/"+dir_name):
        os.mkdir("./8_nebV1V2/"+dir_name)
    
write_vasp("./8_nebV1V2/"+dir_list[0]+"/POSCAR",V1,label="changed",direct=True,)
write_vasp("./8_nebV1V2/"+dir_list[-1]+"/POSCAR",V2,label="changed",direct=True,)
for i in range(1,len(dir_list)-1):
    write_vasp("./8_nebV1V2/"+dir_list[i]+"/POSCAR",img_list[i-1],label="changed",direct=True,)

# neb V2 to V3

In [20]:
V2=read("./6_V2positive/CONTCAR")
V3=read("./7_V3positive/CONTCAR")
V3.positions=move_near(V2.positions,V3.positions,V2.cell)
view(V2)
view(V3)

In [14]:
def neb_rotate(V1,V2,base,ro,n_img=4):
    # V1,V2 ase.Atoms 
    # base int index of atom of rotation center
    # ro int index of atom to be rotated
    # n_img int number of images in neb
    img_list=[]
    for i in range(n_img):
        img_list.append(V1.copy())
        img_list[i].positions=V1.positions+(i+1)/(n_img+1)*(V2.positions-V1.positions)
        
    d1=np.sqrt(np.square(V1[ro].position-V1[base].position).sum())
    d2=np.sqrt(np.square(V2[ro].position-V2[base].position).sum())
    
    for i in range(n_img):
        vec=img_list[i][ro].position-img_list[i][base].position
        vec=vec/np.linalg.norm(vec)*(d1*(i+1)/(n_img+1)+d2*(n_img-i)/(n_img+1))
        img_list[i][ro].position=img_list[i][base].position+vec
    return img_list

In [18]:
img_list=neb_rotate(V2,V3,185,165)
view(img_list[0])

In [21]:
from ase.io.vasp import write_vasp
import os

dir_list=[]
for i in range(n_img+2):
    if i<10:
        dir_name="0"+str(i)
    else:
        dir_name=str(i)
    dir_list.append(dir_name)
    if not os.path.exists("./9_nebV2V3/"+dir_name):
        os.mkdir("./9_nebV2V3/"+dir_name)
    
write_vasp("./9_nebV2V3/"+dir_list[0]+"/POSCAR",V2,label="changed",direct=True,)
write_vasp("./9_nebV2V3/"+dir_list[-1]+"/POSCAR",V3,label="changed",direct=True,)
for i in range(1,len(dir_list)-1):
    write_vasp("./9_nebV2V3/"+dir_list[i]+"/POSCAR",img_list[i-1],label="changed",direct=True,)